In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

/home/kumarsau/venvs/jenv/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


#### sample data generation

In [2]:
# Sample data
symbols = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']
strategies = ['Strategy_A', 'Strategy_B', 'Strategy_C', 'Strategy_D', 'Strategy_E']

# Generate sample data
n = 1000
# columns : Ticker, Strategy, Qty and ea_date
data = {
    'Ticker': np.random.choice(symbols, n),
    'ea_date': [datetime.now() - timedelta(days=random.randint(0, 13)) for _ in range(n)],
    'Strategy': np.random.choice(strategies, n),
    'Qty': np.random.randint(1, 1000, n)
    }

In [3]:
# Create DataFrame
df = pd.DataFrame(data)
# Drop the time part from the date column
df['ea_date'] = df['ea_date'].dt.date

#### format raw data to date indexed dataframe for given dates

In [4]:
def format_dateidx(df, dates, dtcol='ea_date', df_cols = ['Ticker', 'Strategy', 'Qty', 'ea_date']):
    df_list = []
    for date in dates:
        df2  = df[df[dtcol]==date]
        df3 = df2[df_cols]
        df3.columns = pd.MultiIndex.from_product([[date], df_cols])
        df3 = df3.drop(dtcol, axis = 1, level = 1)
        # print(df3)
        df_list.append(df3)
        
    df_res = df_list[0]
    for i in range(len(df_list)-1):
        df_res = pd.concat([df_res, df_list[i+1]], axis=0, ignore_index=True)
        
    df_sorted = df_res.copy()
    for date in dates:
    	df_sorted[date] = df_sorted[date].sort_values(by='Ticker').reset_index(drop=True)
    df_sorted = df_sorted.dropna(axis = 0, how = 'all')
    df_sorted = df_sorted.fillna("") 
    return df_sorted

In [5]:
dates = df['ea_date'].drop_duplicates().tolist()
dates.sort()
n = 5
datesl = [dates[i:i+n] for i in range(0, len(dates), n)]

In [6]:
fdfl = []
for x in datesl:
    fdfl.append(format_dateidx(df, x))

#### dataframe to html format 1 

In [7]:
from IPython.display import display, HTML

In [8]:
def append_table(df, carry_html=None):
    # Convert DataFrame to HTML
    html = df.to_html()
    
    # Create HTML with CSS for scrollable table
    scrollable_html = f"""
    <style>
        .scrollable-table {{
            max-height: 400px;
            overflow-y: scroll;
            overflow-x: scroll;
            display: block;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
        }}
        th, td {{
            border: 1px solid black;
            padding: 5px;
            text-align: left;
        }}
    </style>
    <div class="scrollable-table">
        {html}
    </div>
    """

    if carry_html:
        scrollable_html = carry_html + scrollable_html 
    return scrollable_html

In [9]:
html = '''
    <p style="color:rgba(0,0,0,1);">Hi. some text for you to insert</p>
'''
i = 1
for x in fdfl:
    html += f"<p style=\"color:rgba(0,0,0,1);\">TABLE {i}</p>"
    html = append_table(x, html)
    i+=1

# with open('saving.html', 'w+') as f:
#     f.write(html)

#### dataframe to html format 2

In [11]:
from pretty_html_table import build_table

html = '''
    <p style="color:rgba(0,0,0,1);">Hi. some text for you to insert</p>
'''
i = 1
for x in fdfl:
    html += f"<p style=\"color:rgba(0,0,0,1);\">TABLE {i}</p>"
    i+=1
    html += build_table(x, 'blue_light')

# with open('saving.html', 'w+') as f:
#     f.write(html)

### email

In [12]:
import os
import ssl
import smtplib
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
sender = "ohboy3947@gmail.com"
password = "GMAIL_PASSWORD"
receiver = "saurabhiitd3@gmail.com"

In [ ]:
c = datetime.now()
current_time = c.strftime('%H:%M:%S')
subject = "test format : " + current_time

In [ ]:
msg = MIMEMultipart('alternative')
msg['Subject'] = subject
msg['From'] = sender
msg['To'] = receiver

In [ ]:
msg.attach(MIMEText(html, 'html'))

In [ ]:
context = ssl.create_default_context()
smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context)

In [ ]:
smtp.login(sender, password)

In [ ]:
smtp.sendmail(sender, receiver, msg.as_string())